<h3>1. Defining functions</h3>
<ol style="margin-left: 20px; font-size: 16px;">
    <li>
        <p><strong>import_data()</strong></p>
        <p style="margin-left: 20px; font-size: 14px;">
        <ul>
            <li>Loads data into a TensorFlow dataset</li>
            <li>Defines a normalization layer</li>
            <li>Divides data into train, validation, and test data</li>
        </ul>
        </p>
    </li>
    <li>
        <p><strong>model_train()</strong></p>
        <p style="margin-left: 20px; font-size: 14px;">
        <ul>
            <li>Trains a model with the training and validation data
                </li>
            <li>Saves the model along with its training history</li>
        </ul>
        </p>
    </li>
    <li>
        <p><strong>model_evaluate()</strong></p>
        <p style="margin-left: 20px; font-size: 14px;">
        <ul style="margin-left: 20px; font-size: 14px;">
            <li>Shows training history</li>
            <li>Plots training accuracy and loss</li>
            <li>Evaluates the model on test data</li>
            <li>Shows a classification report</li>
            <li>Shows the confusion matrix</li>
        </ul>
        </p>
    </li>
</ol>


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Resizing, GlobalAveragePooling2D, GlobalMaxPooling2D, Dense, InputLayer, RandomRotation, RandomFlip, RandomTranslation, RandomZoom, BatchNormalization, Rescaling, Dropout, Conv2D, MaxPool2D, Flatten, Normalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.efficientnet import EfficientNetB0

from imblearn.over_sampling import RandomOverSampler
import os
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import zipfile
import itertools
import shutil

def get_dataset(datasets):
    os.environ['KAGGLE_USERNAME'] = 'mohamedadel452'
    os.environ['KAGGLE_KEY'] = 'a6ea873bc8a4c8196d2683d147696840'
    for dataset in datasets:
        !kaggle datasets download -d {dataset}
        with zipfile.ZipFile('/content/' + dataset.split('/')[1] + '.zip', 'r') as zip_ref:
            zip_ref.extractall('/content/' + dataset.split('/')[1])

def convert_to_tflite(saved_model_dir):
  converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
  tflite_model = converter.convert()

  with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

def delete_directory(dirs):
    try:
      for dir in dirs:
        shutil.rmtree(dir)
        print(f"Directory deleted successfully.")
    except Exception as e:
        print(f"Error: {e}")

def import_data(Image_directory, classes):
    chunks = ['train_data','validation_data','test_data']

    batch_size = 128

    train_data = image_dataset_from_directory(Image_directory + '/' + chunks[0], labels = "inferred", label_mode = "int",
          seed=123, shuffle = True, image_size=(128, 128), batch_size=batch_size)

    val_data = image_dataset_from_directory(Image_directory + '/'+ chunks[1], labels = "inferred", label_mode = "int",
      seed=123, shuffle = True, image_size=(128, 128), batch_size=batch_size)

    test_data = image_dataset_from_directory(Image_directory + '/' + chunks[2], labels = "inferred", label_mode = "int",
          seed=123, shuffle = True, image_size=(128, 128), batch_size=batch_size)


    #Train, Test, and Validation Split adjustment
    train_n_batches = tf.data.experimental.cardinality(train_data).numpy()

    val_n_batches = tf.data.experimental.cardinality(val_data).numpy()
    test_n_batches = tf.data.experimental.cardinality(test_data).numpy()

    addition = int(train_n_batches * 0.05)

    val_data = val_data.concatenate(train_data.take(addition))
    test_data = test_data.concatenate(train_data.take(addition))


    #Normalization layer for Basic CNN
    Normalization_layer = Normalization(axis = -1)
    train_image_data = train_data.map(lambda x , y: x)
    val_image_data = val_data.map(lambda x , y: x)
    data_0 = train_image_data.concatenate(val_image_data)

    Normalization_layer.adapt(data_0)

    return train_data, val_data, test_data, classes, Normalization_layer

def model_and_history_dir(model_name, model_n, runs = [0]):

    save_dir = '/content/model_' + str(model_n) + '_' + model_name + '/'

    history_dir = '/content/model_' + str(model_n) + '_' + model_name + '_history/'

    if not os.path.exists(history_dir):
        os.makedirs(history_dir)

    model_history_dir = []

    for run_n in runs:
      model_history_dir.append(history_dir + 'model_' + str(model_n) + '_' + model_name + '_training_history' + '_'  + str(run_n) + '.csv' )

    return save_dir, model_history_dir

def restore_model(save_dir, model):
    if os.path.exists(save_dir):
      model = load_model(save_dir)
    return model

def model_train(data, n_classes, n_epochs, model, model_n, lr = .001, run_n = [0], Normalize_bool = True, pt_model_w = None, tl_mode = 0, bn_train = False, bn_layer = None, pt_model_ftl_perc = None):

    train_data, val_data, test_data, classes, Normalization_layer = data
    image_size = (128,128)
    model_name = model[0]
    pt_model = model[1]

    model = Sequential ([Resizing(*image_size)])

    if Normalize_bool:
      model.add(Normalization_layer)

    model.add(RandomFlip("horizontal_and_vertical"))
    model.add(RandomRotation(0.2))
    model.add(RandomZoom(height_factor=(-0.1, 0.1)))

    if pt_model:

        pt_model = pt_model(include_top=False, weights = pt_model_w, input_shape = (*image_size,3))

        if tl_mode == 0:
            pt_model.trainable = False

        elif tl_mode == 1:
            pt_model.trainable = True
            fine_tune_at = .3 * len(pt_model.layers)
            if pt_model_ftl_perc:
                fine_tune_at = int(pt_model_ftl_perc * len(pt_model.layers))
            for layer in pt_model.layers[:fine_tune_at]:
                layer.trainable = False

        elif tl_mode == 2:
            pt_model.trainable = True

        if bn_layer:
          for layer in pt_model.layers[bn_layer:]:
            if isinstance(layer, BatchNormalization):
              layer.trainable = bn_train
        else:
          for layer in pt_model.layers:
            if isinstance(layer, BatchNormalization):
              layer.trainable = bn_train

        model.add(pt_model)

    else:

        for layer in [
            Conv2D(16, kernel_size = (3,3), input_shape = (*image_size, 3), activation = 'relu', padding = 'same'),
            Conv2D(32, kernel_size = (3,3), activation = 'relu'),
            MaxPool2D(pool_size = (2,2)),
            Conv2D(32, kernel_size = (3,3), activation = 'relu', padding = 'same'),
            Conv2D(64, kernel_size = (3,3), activation = 'relu'),
            MaxPool2D(pool_size = (2,2), padding = 'same'),
        ]:
          model.add(layer)

    for layer in [GlobalAveragePooling2D(),
            Dense(2048, activation='relu'),
            Dense(1024, activation='relu'),
            Dense(n_classes)]:
            model.add(layer)

    learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=1, factor=0.5,
                                            min_lr=0.0001, cooldown=2)

    model.compile(optimizer=Adam(learning_rate = lr, amsgrad=True,),
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=[SparseCategoricalAccuracy(name='accuracy')]
              )


    save_dir, model_history_dir = model_and_history_dir(model_name, model_n, run_n)

    model_history_dir = model_history_dir[0]

    model = restore_model(save_dir, model)

    history = model.fit(train_data,
                        epochs=n_epochs,
                        validation_data=val_data,
                        callbacks=[learning_rate_reduction]
    )

    model.save(save_dir)

    pd.DataFrame(history.history).to_csv(model_history_dir, index=True)

def model_evaluate(data, model_name, model_n, runs = [0]):
    _, _, test_data, classes, _ = data

    checkpointdir, model_history_dir = model_and_history_dir(model_name, model_n, runs)

    model = load_model(checkpointdir)

    histories = []

    for history in runs:
      histories.append( pd.read_csv(model_history_dir[history]) )

    history = pd.concat(histories)
    history = history.reset_index(drop=True)

    test_images , test_labels = zip(*list(test_data.map(lambda x, y: (x, y))))
    test_images = np.concatenate(test_images)
    test_labels = np.concatenate(test_labels)

    #Showing training history
    history_show = history.sort_values(by=['accuracy'], ascending=False).drop(['Unnamed: 0'], axis = 1)
    print('Training history','\n', history_show.head(10),'\n')

    #Evaluating model on test data
    loss_test, acc_test = model.evaluate(test_data, verbose=1)
    print("Test: accuracy = %f  ;  loss = %f" % (acc_test, loss_test))
    print('\n')

    #Classification report
    test_pred = np.argmax( model.predict(test_images), axis = 1)
    print('\n')
    print(classification_report(test_labels, np.around(test_pred, decimals=0)))

    #Plotting training accuracy and loss
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    t = f.suptitle(model_name + ' Performance', fontsize=12)
    f.subplots_adjust(top=0.85, wspace=0.3)

    ax1.plot(history['accuracy'], label='Train Accuracy')
    ax1.plot(history['val_accuracy'], label='Validation Accuracy')
    ax1.set_xticks(np.arange(0, len(history), 5))
    ax1.set_ylabel('Accuracy Value')
    ax1.set_xlabel('Epoch')
    ax1.set_title('Accuracy')
    l1 = ax1.legend(loc="best")

    ax2.plot(history['loss'], label='Train Loss')
    ax2.plot(history['val_loss'], label='Validation Loss')
    ax2.set_xticks(np.arange(0, len(history), 5))
    ax2.set_ylabel('Loss Value')
    ax2.set_xlabel('Epoch')
    ax2.set_title('Loss')
    l2 = ax2.legend(loc="best")

    #Confusion matrix
    confusion_mtx = cm =  confusion_matrix(test_labels, test_pred)
    plt.figure(figsize=(10,7))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Test Confusion matrix')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    print('\n')

model_names = list({'Basic_CNN': None, 'EfficientNetB0': EfficientNetB0}.items())


<h3>2. Model Training and Evaluation</h3>

### Model-4

In [ ]:
#Model_4
datasets = ['mohamedadel452/model-4-data-preprocessed']
Image_directory  = '/content/model-4-data-preprocessed'
classes = {0:"0", 1:"1"}
model = model_names[1]
model_n = 4
n_classes = len(classes.items())

get_dataset(datasets)
data = import_data(Image_directory, classes)
saved_model_dir = model_and_history_dir(model[0], str(model_n), runs = [0])[0]
print(saved_model_dir)
model_train(data, n_classes, n_epochs = 10, model = model, model_n = model_n, lr = .001, run_n = [0], Normalize_bool = False, pt_model_w = 'imagenet', tl_mode = 0, bn_train = False, bn_layer = None)
model_train(data, n_classes, n_epochs = 20, model = model, model_n = model_n, lr = .001, run_n = [1], Normalize_bool = False, pt_model_w = 'imagenet', tl_mode = 1, bn_train = False, pt_model_ftl_perc = .3)

model_evaluate(data, model[0], runs = [0, 1])
convert_to_tflite(saved_model_dir)
shutil.make_archive('model_output', 'zip', saved_model_dir)


Dataset URL: https://www.kaggle.com/datasets/mohamedadel452/model-4-data-preprocessed
License(s): unknown
 99% 519M/525M [00:03<00:00, 199MB/s]
100% 525M/525M [00:03<00:00, 180MB/s]
Found 167750 files belonging to 2 classes.
Found 17060 files belonging to 1 classes.
Found 8916 files belonging to 1 classes.
/content/model_4_EfficientNetB0/
16705208/16705208 [==============================] - 0s 0us/step
Epoch 1/10
1311/1311 [==============================] - 154s 107ms/step - loss: 0.0045 - accuracy: 0.9986 - val_loss: 43.9117 - val_accuracy: 0.3287 - lr: 0.0010
Epoch 2/10
1311/1311 [==============================] - 135s 103ms/step - loss: 0.0011 - accuracy: 0.9997 - val_loss: 44.7133 - val_accuracy: 0.3282 - lr: 0.0010
Epoch 3/10
1311/1311 [==============================] - ETA: 0s - loss: 8.7592e-04 - accuracy: 0.9997
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
1311/1311 [==============================] - 139s 106ms/step - loss: 8.7592e-04 - accuracy: 